# AWS IoT Greengrass Workshop

[Requirements for Greengrass on an EC2 instance](https://docs.aws.amazon.com/greengrass/latest/developerguide/module1.html#setup-filter.ec2)

**This workshop is currently broken when starting the greengrass devices on the EC2 instance.**

### Initialize Environment

In [ ]:
import boto3
import sys
import os
import json
import base64
import project_path # path to helper methods

from lib import workshop
from botocore.exceptions import ClientError
project_name = 'iot-greengrass-workshop'

ec2_client = boto3.client('ec2')
ec2 = boto3.resource('ec2')

gg = boto3.client('greengrass')
iot = boto3.client('iot')
cfn = boto3.client('cloudformation')

session = boto3.session.Session()
region = session.region_name

stack_name = 'GGDEC2InstanceStack'
instance_size = 't3.medium'

### [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing our data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
bucket = workshop.create_bucket_name('gg-')
session.resource('s3').create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
print(bucket)

### [Getting Started with Greengrass](https://docs.aws.amazon.com/greengrass/latest/developerguide/gg-gs.html)

In order to help simplify the setup process we will be using a setup utility available on Github called [aws-greengrass-group-setup
](https://github.com/awslabs/aws-greengrass-group-setup). This allows us to take a file based approach to the entire Greengrass group for our health tracker application. 

In [ ]:
%cd health_tracker

In [ ]:
!source activate JupyterSystemEnv
!pip install -r requirements.txt

### [Create Greengrass Core](https://docs.aws.amazon.com/greengrass/latest/developerguide/module2.html)

In [ ]:
!cp tracker/cfg_template.json tracker/cfg.json

In [ ]:
%run ./group_setup.py create-core --thing-name tracker-core --config-file tracker/cfg.json --cert-dir tracker/certs

### Validate creation
Once the `group_setup.py` runs successfully it will have create the `tracker-core` IoT thing, the client certificate, and policy for the device. To validate in the AWS Console click the link below.

In [ ]:
print('https://{0}.console.aws.amazon.com/iot/home?region={0}#/thinghub'.format(region))

### Dowload root certs

In [ ]:
!curl -o root-ca.pem https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem
!echo tracker/certs tracker/ggd/certs | xargs -n 1 cp root-ca.pem
!rm root-ca.pem

### [Create IoT Core Devices](https://docs.aws.amazon.com/iot/latest/developerguide/configure-iot.html)

Devices connected to AWS IoT are represented by `_things_` in the registry. The registry allows you to keep a record of all of the devices that are connected to your AWS IoT account.

In [ ]:
%run ./group_setup.py create-devices --thing-names '[hr_ggd,heartbeat_ggd,web_ggd]' --config-file tracker/cfg.json --cert-dir tracker/ggd/certs


In [ ]:
%run ./group_setup.py create-devices --thing-names '[tracker_brain]' --config-file tracker/cfg.json \
--cert-dir tracker/ggd/certs --append True --cloud_sync True


### [Create Greengrass Lambda Functions](https://docs.aws.amazon.com/greengrass/latest/developerguide/config-lambda.html)

In [ ]:
%run ./lambda_setup.py create lambda/TrackerBrain/cfg_lambda.json

In [ ]:
%run ./lambda_setup.py create lambda/TrackerErrorDetector/cfg_lambda.json

### Associate IoT Core devices to Lambda Functions

In [ ]:
%run ./group_setup.py associate-lambda ./tracker/cfg.json ./lambda/TrackerBrain/cfg_lambda.json

In [ ]:
%run ./group_setup.py associate-lambda ./tracker/cfg.json ./lambda/TrackerErrorDetector/cfg_lambda.json

### Deploy Greengrass Core

In [ ]:
%run ./group_setup.py create tracker ./tracker/cfg.json --group_name tracker

In [ ]:
%run ./group_setup.py deploy ./tracker/cfg.json

In [ ]:
!cat tracker/cfg.json

### Create the Greengrass config.json for the Tracker

In [ ]:
%%writefile gg_config.json

{
   "coreThing": {
       "caPath": "root-ca.pem",
       "certPath": "tracker-core.pem",
       "keyPath": "tracker-core.prv",
       "thingArn": "arn:aws:iot:{{region}}:{{aws_account_id}}:thing/master-core",
       "iotHost": "{{iot_endpoint}}.iot.{{region}}.amazonaws.com",
       "ggHost": "greengrass.iot.{{region}}.amazonaws.com",
       "keepAlive": 600
   },
   "runtime": {
       "cgroup": {
           "useSystemd": "yes"
       }
   }
}

### Zip device software

In [ ]:
!zip -r tracker.zip tracker/

### [Upload to S3](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html)

Next, we will upload the json file created above to S3 to be used later in the workshop.

[s3.upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_file) boto3 documentation

In [ ]:
file_name = 'tracker.zip'
session.resource('s3').Bucket(bucket).Object(os.path.join('device', file_name)).upload_file(file_name)

### [Create VPC](https://docs.aws.amazon.com/vpc/index.html) 

In order to simulate a Greengrass device on an EC2 instance we will create a new VPC with a public subnet by running the code below. As you can see to make a subnet public an Internet Gateway is attached to the VPC and a routing table is created with and entry to route all traffic at `0.0.0.0/0` to the Internet gateway. We will store the VPC and Subnet Id's to be used later in the notebook.

In [ ]:
vpc, subnet1, subnet2 = workshop.create_and_configure_vpc()

In [ ]:
vpc_id = vpc.id
subnet1_id = subnet1.id
subnet2_id = subnet2.id
print(vpc_id)
print(subnet1_id)
print(subnet2_id)

### [Create Security Group for EC2 instance](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_SecurityGroups.html)

A security group acts as a virtual firewall for your instance to control inbound and outbound traffic. When you launch an instance in a VPC, you can assign up to five security groups to the instance. Security groups act at the instance level, not the subnet level. Therefore, each instance in a subnet in your VPC could be assigned to a different set of security groups. If you don't specify a particular group at launch time, the instance is automatically assigned to the default security group for the VPC.

The security group will open ports `80` and `8883` respectively for HTTP and MQTT access.

[ec2_client.create_security_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.create_security_group)

In [ ]:
sec_group = ec2_client.create_security_group(
    Description='Security Group for EC2 instance acting as IoT Greengrass device',
    GroupName=project_name+'-sg',
    VpcId=vpc_id
)

sec_group_id=sec_group["GroupId"]
print(sec_group_id)

### [Authorizing Inbound Traffic for Your Linux Instances](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/authorizing-access-to-an-instance.html)

Security groups enable you to control traffic to your instance, including the kind of traffic that can reach your instance. For example, you can allow computers from only your home network to access your instance using SSH. If your instance is a web server, you can allow all IP addresses to access your instance using HTTP or HTTPS, so that external users can browse the content on your web server.

[ec2_client.authorize_security_group_ingress](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.authorize_security_group_ingress)

In [ ]:
data = ec2_client.authorize_security_group_ingress(
    GroupId=sec_group_id,
    IpPermissions=[
        {
         'IpProtocol': 'tcp',
         'FromPort': 8883,
         'ToPort': 8883,
         'IpRanges': [
            {
                'CidrIp': '0.0.0.0/0',
                'Description': 'MQTT access'
            },
          ]
        },
        {        
         'IpProtocol': 'tcp',
         'FromPort': 80,
         'ToPort': 80,
         'IpRanges': [
            {
                'CidrIp': '0.0.0.0/0',
                'Description': 'HTTP access'
            },
          ]
        }
    ]
)

print(data)

### Get latest [Amazon Linux AMI](https://aws.amazon.com/amazon-linux-ami/) in the region

The Amazon Linux AMI is a supported and maintained Linux image provided by Amazon Web Services for use on Amazon Elastic Compute Cloud (Amazon EC2). It is designed to provide a stable, secure, and high performance execution environment for applications running on Amazon EC2. It supports the latest EC2 instance type features and includes packages that enable easy integration with AWS. Amazon Web Services provides ongoing security and maintenance updates to all instances running the Amazon Linux AMI. The Amazon Linux AMI is provided at no additional charge to Amazon EC2 users.

We will lookup the latest AMI version of the Amazon Linux OS to be used for the EC2 instance.

In [ ]:
ami = workshop.get_latest_amazon_linux()
print(ami)

### Launch EC2 instance and install [Greengrass](https://aws.amazon.com/greengrass/)

The UserData section of the EC2 instance launch includes everything needed to configure and install Greengrass on the EC2 instance. View the UserData below to get an understanding of what's involved to configure Greengrass on devices. [Greengrass Core downloads](https://docs.aws.amazon.com/greengrass/latest/developerguide/what-is-gg.html#gg-core-download-tab) link provides the available devices and OS's available.

Before running the EC2 create instance cell change the `REGION=<CHOOSE REGION>` to the region you are running in. i.e. `REGION=us-west-2`

In [ ]:
!cat greengrass-device.yaml

### Upload [CloudFormation](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/GettingStarted.html) template

In the interest of time we will leverage CloudFormation to launch an EC2 instance that will install the appropriate Greengrass software to mock an edge device.

In [ ]:
file_name = 'greengrass-device.yaml'
session.resource('s3').Bucket(bucket).Object(os.path.join('cfn', file_name)).upload_file(file_name)


### Create CloudFormation Stack for Greengrass EC2 instance

In [ ]:
cfn_template = 'https://s3-{0}.amazonaws.com/{1}/cfn/{2}'.format(region, bucket, file_name)
print(cfn_template)
response = cfn.create_stack(
    StackName=stack_name,
    TemplateURL=cfn_template,
    Capabilities = ["CAPABILITY_NAMED_IAM"],
    Parameters=[
        {
            'ParameterKey': 'InstanceType',
            'ParameterValue': instance_size
        },
        {
            'ParameterKey': 'Subnet',
            'ParameterValue': subnet1_id
        },
        {
            'ParameterKey': 'ImageId',
            'ParameterValue': ami
        },
        {
            'ParameterKey': 'SG',
            'ParameterValue': sec_group_id
        },
        {
            'ParameterKey': 'S3BucketName',
            'ParameterValue': bucket
        }
    ]
)

print(response)

### Wait for CloudFormation template to complete

In [ ]:
waiter = cfn.get_waiter('stack_create_complete')
waiter.wait(
    StackName=stack_name
)

print('The wait is over for {0}'.format(stack_name))

### Get EC2 instance id from template

We will get the instance id of the EC2 instance to use in the next section from the `Output` section of the CloudFormation template.

In [ ]:
response = cfn.describe_stacks(
    StackName=stack_name
)

ec2_instance_id = response['Stacks'][0]['Outputs'][0]['OutputValue']
print(ec2_instance_id)

### Shell access to EC2 instance with [Systems Manager Session Manager](https://docs.aws.amazon.com/systems-manager/latest/userguide/session-manager.html)

Session Manager is a fully managed AWS Systems Manager capability that lets you manage your Amazon EC2 instances through an interactive one-click browser-based shell or through the AWS CLI. Session Manager provides secure and auditable instance management without the need to open inbound ports, maintain bastion hosts, or manage SSH keys. Session Manager also makes it easy to comply with corporate policies that require controlled access to instances, strict security practices, and fully auditable logs with instance access details, while still providing end users with simple one-click cross-platform access to your Amazon EC2 instances.

Execute the cell below and open the link in a new tab. You will now start shell access into the EC2 instance to complete the workshop. Select your instance id as noted above and click the `Start Session` button. 

![Session Manager](../../docs/assets/images/session_manager.png)

In [ ]:
print('https://{0}.console.aws.amazon.com/systems-manager/session-manager/start-session?region={0}'.format(region))

## Get shell access into the EC2 instance and run the final commands to start the GG Core and Devices

### To start device operations of the Master host `ec2 instance`
1. **Start GG Core** -- in the `ec2 instance` terminal execute:
    ```bash
    cd /greengrass/ggc/packages/<version> 
    sudo ./greengrassd start
    ```
1. **Start GG Devices** -- in the `ec2 instance` terminal execute:
    ```bash
    cd ~/groups/tracker
    ./start_tracker.sh
    ```
    
After starting the tracker devices, to determine success you should see three 
entries in the list of processes, similar to the following:
```
ec2 instance$ screen -ls
There are screens on:
        2540.web        (Detached)
        2537.heartbeat  (Detached)
        2534.heartrate  (Detached)
3 Sockets in /var/run/screen/S-root.
```
To view the output of any of the Greengrass Devices attach to the 
`screen` by using the command `screen -r <pid>`. Example that 
re-attaches to the `web` device process in the above list:
```
screen -r 8281
```
:warning: Remember to detach from the screen using `Ctrl-A, D` **not** `Ctrl-C`. 
Using `Ctrl-C` will exit the process being viewed.

If the `hr` device started successfully you should see messages arriving to the IoT core through the MQTT topic.

### To stop device operations of the Tracker host
1. **Stop GG Devices** -- in `ec2 instance` Terminal execute:
    ```bash
    cd ~/groups/tracker
    ./stop_tracker.sh
    ```

### Monitor messages from GG device in console

In [ ]:
print('https://{}.console.aws.amazon.com/iot/home?region={}#/heart/beat'.format(region))

## Cleanup

In [ ]:
response = cfn.delete_stack(StackName=stack_name)

In [ ]:
waiter = cfn.get_waiter('stack_delete_complete')
waiter.wait(
    StackName=stack_name
)

print('The wait is over for {0}'.format(stack_name))

In [ ]:
%run ./group_setup.py clean-all ./tracker/cfg.json
# create a delete for the Lambda functions

In [ ]:
workshop.vpc_cleanup(vpc_id)